# Тестирование предобработки данных

## Обработка данных в датасете

In [1]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

In [2]:
from src.data_utils import MamsClaraDataset, debug_clara_batch, stack_batch

In [ ]:
from transformers import AutoTokenizer

model_id = "microsoft/Phi-3.5-mini-instruct"
NUM_MEM_TOKENS = 4

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

mem_tokens = [f"[M{i}]" for i in range(NUM_MEM_TOKENS)]
tokenizer.add_special_tokens({"additional_special_tokens": mem_tokens})

tokenizer.padding_side = 'right' 

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [5]:
# check special tokens
print(tokenizer.tokenize("[M0] [M1] hello"))
print(tokenizer.convert_tokens_to_ids(["[M0]", "[M1]"]))

['[M0]', '▁▁', '[M1]', '▁', '▁hello']
[32011, 32012]


In [6]:
ds = MamsClaraDataset("../data/train.xml", tokenizer)
print("Raw sentences:", len(ds.raw_data))
print("Flattened samples:", len(ds))

print(ds.raw_data[0])
print(ds[0].keys())
print(ds[0]["task"], tokenizer.decode(ds[0]["dec_input_ids"]))

Raw sentences: 4297
Flattened samples: 19780
{'text': 'The decor is not special at all but their food and amazing prices make up for it.', 'aspects': [{'term': 'decor', 'polarity': 'negative', 'from': 4, 'to': 9}, {'term': 'food', 'polarity': 'positive', 'from': 42, 'to': 46}, {'term': 'prices', 'polarity': 'positive', 'from': 59, 'to': 65}]}
dict_keys(['enc_input_ids', 'enc_mask', 'dec_input_ids', 'dec_mask', 'labels', 'task'])
rec [M0]  [M1]  [M2]  [M3]  Restore text:  The decor is not special at all but their food and amazing prices make up for it.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><

In [7]:
print("pad_token:", tokenizer.pad_token, tokenizer.pad_token_id)
print("eos_token:", tokenizer.eos_token, tokenizer.eos_token_id)

pad_token: <|endoftext|> 32000
eos_token: <|endoftext|> 32000


In [8]:
item = ds[0]
ids = item["dec_input_ids"]
mask = item["dec_mask"].bool()
print(item["task"], tokenizer.decode(ids[mask], skip_special_tokens=False))

rec [M0]  [M1]  [M2]  [M3]  Restore text:  The decor is not special at all but their food and amazing prices make up for it.<|endoftext|>


In [9]:
batch = stack_batch([ds[0], ds[1], ds[2]])
debug_clara_batch(batch, tokenizer)


DIAGNOSTIC REPORT FOR BATCH (Batch Size: 3)

--- SAMPLE 0 | TASK: REC ---
[Encoder Input]: The decor is not special at all but their food and amazing prices make up for it. [M0]  [M1]  [M2]  ... endoftext|><|endoftext|><|endoftext|><|endoftext|>
  > Memory Tokens Detected: ['[M0]', '[M1]', '[M2]', '[M3]']
  > Padding tokens count: 101
[Decoder Input]: [M0]  [M1]  [M2]  [M3]  Restore text:  The decor is not special at all but their food and amazing prices make up for it.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|end

## Формирование батчей

In [10]:
task_weights = {"rec": 1.0, "ext": 1.0, "reason": 0.5} # decrease frequency of 'reason' task
weights = [task_weights[s['task']] for s in ds.samples]

In [11]:
import torch 

# create sampler to shuffle data
sampler = torch.utils.data.WeightedRandomSampler(
    weights=weights, 
    num_samples=len(ds), 
    replacement=True
)

In [14]:
# transfer to dataloader with sampler
train_loader = torch.utils.data.DataLoader(
    ds, 
    batch_size=16, 
    sampler=sampler, 
    num_workers=12
)

In [16]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [18]:
from collections import Counter

# get first 100 batches and count tasks on types
counts = Counter()
for i, batch in enumerate(train_loader):
    counts.update(batch['task'])
    if i > 100: break

print("Frequency of tasks in train dataset (based on 100 batches):", counts)

Frequency of tasks in train dataset (based on 100 batches): Counter({'reason': 631, 'rec': 509, 'ext': 492})


In [20]:
for i, batch in enumerate(train_loader):
    print(batch)
    break

{'enc_input_ids': tensor([[ 1334, 10325,  3512,  ..., 32000, 32000, 32000],
        [  360,  3993,   471,  ..., 32000, 32000, 32000],
        [ 9208,   368, 29892,  ..., 32000, 32000, 32000],
        ...,
        [  450, 25005,   338,  ..., 32000, 32000, 32000],
        [25453,   471,  3431,  ..., 32000, 32000, 32000],
        [ 1987,   278,  6143,  ..., 32000, 32000, 32000]]), 'enc_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'dec_input_ids': tensor([[32011,   259, 32012,  ..., 32000, 32000, 32000],
        [32011,   259, 32012,  ..., 32000, 32000, 32000],
        [32011,   259, 32012,  ..., 32000, 32000, 32000],
        ...,
        [32011,   259, 32012,  ..., 32000, 32000, 32000],
        [32011,   259, 32012,  ..., 32000, 32000, 32000],
        [32011,   259, 32012,  ..., 32000, 32000, 32000]]), 'dec_mas